In [1]:
from dataclasses import dataclass
import ctypes
import os, sys
from enum import IntEnum

sys.path.append( '..' )
from datagenDV import *

In [2]:
class MEM_REGIONS_E(IntEnum):
    ROM = 1
    RAM = 2
    CD = 3
    FLOPPY = 4
    MAGNET_TAPE = 5
    RAND = 6

c_enums = [MEM_REGIONS_E]

@dataclass
class FrameData(YAMLParamsBase, ctypes.Structure):
    width : ctypes.c_uint32
    height : ctypes.c_uint32
    chksum : ctypes.c_uint32
    padding : ctypes.c_uint32
    location : MEM_REGIONS_E

FrameData.generate_hfields()
c_structs = [FrameData]


In [3]:
if not os.path.exists('out'):
  os.mkdir('out')
with open("out/frame.h", 'w') as fh:
    ctypes_helper.write_enum_type_defs(fh, enums=[MEM_REGIONS_E])
    ctypes_helper.write_ctype_structs(fh, ctype_structs=[FrameData], use_hfields=True)
myFrame = FrameData( width=0xABCD, height=0x1234, chksum=0x9876, padding=0x5432, location=MEM_REGIONS_E.RAND)

In [4]:
!cat out/frame.h


typedef enum {
  ROM = 1,
  RAM = 2,
  CD = 3,
  FLOPPY = 4,
  MAGNET_TAPE = 5,
  RAND = 6
} MEM_REGIONS_E;

typedef struct FrameData {
   unsigned int    width;
   unsigned int    height;
   unsigned int    chksum;
   unsigned int    padding;
   MEM_REGIONS_E   location;
} FrameData, *PFrameData;



In [5]:
# FrameData_ctype = create_ctype_class('FrameData_ctypes', ctypes.Structure,
#                            convert_hfields_to_fields(FrameData._hfields_))

# myFrame_dict = {dc_field.name: getattr(myFrame,dc_field.name) for dc_field in dataclasses.fields(myFrame) }
# myFrame_ctype = FrameData_ctype(**myFrame_dict)
# with open("out/framedata_output.bin", 'wb') as fh:
#     fh.write(myFrame_ctype)
    
# !hexdump -x out/framedata_output.bin

In [6]:
ctypes_helper.write_ctype_obj_binary(myFrame, "out/framedata_output.bin")
!hexdump -x out/framedata_output.bin

0000000    abcd    0000    1234    0000    9876    0000    5432    0000
0000010    0006    0000                                                
0000014


In [7]:
c_test_code = """
#include <stdio.h>
#include <stdlib.h>
#include "frame.h"
int main()
{
    FILE* frame_fh = fopen("framedata_output.bin", "rb");
    FrameData* myFrame = malloc(sizeof(FrameData ));
    fread(myFrame, sizeof(FrameData), 1, frame_fh);
    printf("width 0x%x height 0x%x chksum 0x%x padding 0x%x location 0x%x\\n", 
      myFrame->width, myFrame->height, myFrame->chksum, myFrame->padding,  myFrame->location);
    fclose(frame_fh);
}
"""

with open("out/c_test.c", 'w') as fh:
    fh.write(c_test_code)

In [8]:
!gcc out/frame.h  out/c_test.c -o  out/c_test_exe

In [9]:
!cd out && c_test_exe

width 0xabcd height 0x1234 chksum 0x9876 padding 0x5432 location 0x6
